# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.



In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv('../output_data/cities.csv', index_col = False)
city_weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Make 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["Lat", "Lng"]].astype(float)
humidity = city_weather_df["Humidity"].astype(float)

# Set dimensions for graph
figure_layout = {
     'width' : '1000px',
     'height':'400px',
     'border':'1px solid black',
     'padding':'1px',
     'margin':'0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()

# Create Heatmap layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down Melissa's best weather conditions for vacationing in
# A max temperature lower than 75 degrees, but higher than 55.
# Wind speed less than 10 and moody clouds ;)
best_temp = city_weather_df ['Max Temp'] < 75
best_temp = city_weather_df ['Max Temp'] > 55
best_wind = city_weather_df ['Wind Speed'] < 10
best_cloudiness = city_weather_df ['Cloudiness'] > 60
best_fog = city_weather_df ['Humidity'] > 60
my_weather = best_temp & best_wind & best_cloudiness & best_fog

# Extra: Drop the rows with nulls
weirdo_vacations = city_weather_df[my_weather]
weirdo_vacations = weirdo_vacations.dropna()
weirdo_vacations = weirdo_vacations.reset_index()
weirdo_vacations.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
1,23,23,maragogi,97,BR,1558378610,69,-9.01,-35.22,83.94,9.42
2,45,45,rosita,72,NI,1558378762,71,13.92,-84.40,84.12,4.41
3,48,48,port alfred,67,ZA,1558378763,69,-33.59,26.89,61.00,8.01
4,91,91,bella vista,62,AR,1558378771,80,-28.51,-59.05,75.66,7.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel df and add a column for hotel name
hotel_df = pd.DataFrame(weirdo_vacations, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Hotel Name'])
hotel_df.head()

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,avarua,CK,-21.21,-159.78,71.60,NaN
1,maragogi,BR,-9.01,-35.22,83.94,NaN
2,rosita,NI,13.92,-84.40,84.12,NaN
3,port alfred,ZA,-33.59,26.89,61.00,NaN
4,bella vista,AR,-28.51,-59.05,75.66,NaN


In [7]:
counter = 0

for index, row in hotel_df.iterrows():

    # Set parameters to search for hotels with 5000 meters.
    params = {"location": f"{row['Lat']}, {row['Lng']}", "radius": 5000, "types": "lodging",
              "keyword": "Hotel", "key": g_key}
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a json response with the parameters above
    response = requests.get(base_url, params=params)
    
    # convert to json and print them easy to read
    response_j = response.json()
    print(json.dumps(response_j, indent=4, sort_keys=True))
    
    results = response_j['results']
    
    # Name the hotel if you get a positive result
    if len(results) > 0:
        name = response_j['results'][0]['name']
        hotel_df.iloc[counter, -1] = name
    counter = counter + 1

hotel_weirdo_df = hotel_df
hotel_weirdo_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.25519,
                    "lng": -159.72975
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.25376987010728,
                        "lng": -159.7282333701073
                    },
                    "southwest": {
                        "lat": -21.25646952989272,
                        "lng": -159.7309330298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Muri Beach Club Hotel Rarotonga",
            "photos": [
                {
                    "height": 2184,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/1070437177611626902

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKdLsrBSyh94W89A_OMm_GUl3zm4XT0baG_v6PRSU7wj-LP49M59pM9vp90_gGsJgwCr3b5beHx326FjeGTM6ZqmxONYHkcKp605FwLiJUuAZLfZFVTHB3yNIfxMG5f9OAMrM5ZqCx_R7hws3mtGlotH5hn_6-hXEOubTY_ktke3Na5kczPdOQooIwRTNmlsBS3x_yMxk43IOa5gc6LPU98_GsKB32klqj5nw3TQC7fuhJDNyeW87zopIdWsoMd6SV3SNyJxjTy1MAOZBmp_-JZuIxe37qBGaaAnr9Km6Q6YXCKflJ5kgdGOGpsuz_snPdb8zHRDjBVlB0Z38zXLxE2SMa8rqp1wczvgg-h4uVFVgR_WCU3a_m_-TIW5_EOJ504JBRxOGgu1Tmhe3ZWfc1jeH1Y8WGF-L6Z_numJnmxkzSCg57uGfU4nXNlhGrlvA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -9.007587899999999,
                    "lng": -35.2169647
                },
                "viewport": {
                    "northeast": {
                        "lat": -9.006226420107279,
                        "lng": -35.21563177010728
                    },
                    "southwest": {
                        "lat":

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 13.9240661,
                    "lng": -84.3969408
                },
                "viewport": {
                    "northeast": {
                        "lat": 13.92536857989272,
                        "lng": -84.39563717010728
                    },
                    "southwest": {
                        "lat": 13.92266892010728,
                        "lng": -84.39833682989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Terciopelo",
            "photos": [
                {
                    "height": 3456,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/118161517274619893459\">Marlon Mal

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.5904314,
                    "lng": 26.8858279
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.58916472010728,
                        "lng": 26.88700332989272
                    },
                    "southwest": {
                        "lat": -33.59186437989273,
                        "lng": 26.88430367010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "MyPond Hotel",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/111925452985229144220\">Rippen</a>"
  

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -28.5020874,
                    "lng": -59.0455484
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.50077742010727,
                        "lng": -59.04426362010728
                    },
                    "southwest": {
                        "lat": -28.50347707989271,
                        "lng": -59.04696327989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel R\u00edo Arriba",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 4000,
                    "html_attributions": [
                   

...

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 2.0344447,
                    "lng": 45.3033553
                },
                "viewport": {
                    "northeast": {
                        "lat": 2.035890029892722,
                        "lng": 45.30452867989273
                    },
                    "southwest": {
                        "lat": 2.033190370107278,
                        "lng": 45.30182902010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Jowhara International Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2448,
                    "html_attributions": [
                    

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 54.2363749,
                    "lng": -125.771098
                },
                "viewport": {
                    "northeast": {
                        "lat": 54.23759042989272,
                        "lng": -125.7700104201073
                    },
                    "southwest": {
                        "lat": 54.23489077010728,
                        "lng": -125.7727100798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Key-oh Lodge",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 3872,
                    "html_attributions": [
                        "<a hre

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKSVEgCryosi_swaOqtpTw-0dlIQA46fXRBuUjG-rzbMUT9dfZyiAwXVn_abQPQXyDMfmGaO7KqZpE1vLdNRRFahZuTp_kmqAm9Vt-JQ7ms0bRTqgr1hYnFOj2ILqjGIKi0jU2tisKzAPDFoOyRrhVXE-8MQKeOdLJYtsknBcVguIc5hkEGrw_EOQ8THUXivS9HfyHZBMTqlV05qQ7cEfC0uH513JV29huU31rQ4ULPVinfBPFJ8g_apskv2c1BE-UfDjpRFZ8PxZ3tYdi0ZMQSUdrrU9jF-MD7WYAHRr-1sdN8y9mY9rvsMss65xIdIPw2y6RcQYNA9yqHkEPFIvMyUqV-g_0rIb1dFDb95SGRnuviY0WczZsD2N37yJT1b8m8fz5tpaikfOEfckJgKEX3i0CRe4OHrIbZpSnH7ACMWaOnE2aYiJloWCncGuZJ8A",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -0.9610782999999999,
                    "lng": 100.3541281
                },
                "viewport": {
                    "northeast": {
                        "lat": -0.9597177701072778,
                        "lng": 100.3554849798927
                    },
                    "southwest": {
                        "lat"

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKrrZX2QPvmtlfcJSTFuA0pfrYBenEOdCmcDuJJqIvlRnkEIwP-kRKeVLV46f68yKEiAS5YBJNE4GfB4ToWnxi9_pR0Ut2GK4rAKmBJPDnVO8PixQ8c7SIMyKCf3tHJBEKwPXhQ4m4CMvqtXyglFPbB9lf5uYPAjonFKUdB7yoc7cbYLSk6ksAepqEjeh6zSbCxQ0CuSlMFLM6wt2ErtDA7A5NK4QLqJ7IcHeCm_qwwKib6z4LWKG_BoemskRoIv7nuGYlDxIVL3Kvq7h8gk6H_lmYyVefQ_O6eqQnGXFg1mb0DJ4MAbD9glwSKpFEMTa5clfTFYhPlg0tkd3c4VK_N6ta0DHTAEV5hV0YYEeyUB4G90u7beqmB77GRwpgV_3FyHTiOS9fJ-HiY025qSoCnjwXzWyOBrkpCcZVT27A0EnPe3o-aF1Wu3nxbdAFjuw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -0.8767149,
                    "lng": 119.8361206
                },
                "viewport": {
                    "northeast": {
                        "lat": -0.8754635201072778,
                        "lng": 119.8372366298928
                    },
                    "southwest": {
                        "lat": -0.8781

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,avarua,CK,-21.21,-159.78,71.60,Muri Beach Club Hotel Rarotonga
1,maragogi,BR,-9.01,-35.22,83.94,POUSADA DOS CABANOS
2,rosita,NI,13.92,-84.40,84.12,Hotel Terciopelo
3,port alfred,ZA,-33.59,26.89,61.00,MyPond Hotel
4,bella vista,AR,-28.51,-59.05,75.66,Hotel Río Arriba
...,...,...,...,...,...,...
82,malindi,KE,-3.22,40.12,77.00,Driftwood Beach Club Ltd
83,mogadishu,SO,2.04,45.34,80.88,Jowhara International Hotel
84,burns lake,CA,54.23,-125.76,56.40,Key-oh Lodge
85,padang,ID,-0.92,100.36,81.24,Yani homestay /penginapan/hostel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_weirdo_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Plot marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))